In [1]:
from pathlib import Path
import os
from sklearn.model_selection import train_test_split

# Modelling and Forecasting
# ==============================================================================
from lightgbm import LGBMRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg

# benchmark stuff
from buildings_bench.data import load_pandas_dataset, benchmark_registry
from buildings_bench.evaluation.managers import DatasetMetricsManager
from buildings_bench import BuildingTypes

import torch


lag = 168

/projects/foundation/pemami/conda/foundation/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

In [3]:
dataset_path = Path('/projects/foundation/eulp/BuildingsBench/')

In [4]:
metrics_manager = DatasetMetricsManager()

benchmark_registry = ['smart']

In [5]:
dataset = 'smart'

In [6]:
dataset_generator = load_pandas_dataset('smart', dataset_path)

In [7]:
dataset_generator.building_type

'residential'

In [8]:
building_types_mask = (BuildingTypes.RESIDENTIAL_INT * torch.ones([1,24,1])).bool()

In [9]:
for building_name, bldg_df in dataset_generator:
    if building_name == 'HomeB':
        break

In [10]:
bldg_df

,power,hour_x,hour_y,month_x,month_y,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,...,month_of_year_5,month_of_year_6,month_of_year_7,month_of_year_8,month_of_year_9,month_of_year_10,month_of_year_11,month_of_year_12,weekend_False,weekend_True
timestamp,,,,,,,,,,,,,,,,,,,,,
2014-01-01 01:00:00,0.480605,0.258819,0.965926,5.000000e-01,0.866025,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
2014-01-01 02:00:00,0.648437,0.500000,0.866025,5.000000e-01,0.866025,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
2014-01-01 03:00:00,0.297454,0.707107,0.707107,5.000000e-01,0.866025,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
2014-01-01 04:00:00,0.564292,0.866025,0.500000,5.000000e-01,0.866025,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
2014-01-01 05:00:00,0.421209,0.965926,0.258819,5.000000e-01,0.866025,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-31 20:00:00,0.513937,-0.866025,0.500000,-2.449294e-16,1.000000,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
2016-12-31 21:00:00,0.598407,-0.707107,0.707107,-2.449294e-16,1.000000,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
2016-12-31 22:00:00,0.391207,-0.500000,0.866025,-2.449294e-16,1.000000,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True


In [11]:
bldg_df = bldg_df.tail(16000)

In [12]:
bldg_train, bldg_test = train_test_split(bldg_df, test_size=0.25, shuffle=False)

In [13]:
forecaster = ForecasterAutoreg(
        regressor        = LGBMRegressor(max_depth=-1, n_estimators=100, n_jobs=24),
        lags             = lag
    )
forecaster.fit(
    y               = bldg_train['power'],
    exog            = bldg_train[[key for key in bldg_train.keys() if key != 'power']]
)
        

/projects/foundation/pemami/conda/foundation/lib/python3.9/site-packages/skforecast/utils/utils.py:346: DataTypeWarning: `exog` may contain only `int`, `float` or `category` dtypes. Most machine learning models do not allow other types of values. Fitting the forecaster may fail. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=ValueTypesExogWarning)
  warnings.warn(


In [14]:
print(f'fit lightgbm to {building_name} training start {bldg_train.index[0]} end {bldg_train.index[-1]}')

fit lightgbm to HomeB training start 2015-03-06 09:00:00 end 2016-07-18 08:00:00


In [15]:
metrics_manager.add_building_to_dataset_if_missing(
    dataset, f'{building_name}')

pred_days = (len(bldg_test) - lag - 24) // 24

#maes = []
for i in range(pred_days):

    seq_ptr =lag + 24 * i

    last_window  = bldg_test.iloc[seq_ptr - lag : seq_ptr]
    ground_truth = bldg_test.iloc[seq_ptr : seq_ptr + 24]

    predictions = forecaster.predict(
        steps       = 24,
        last_window = last_window['power'],
        exog        = ground_truth[[key for key in bldg_test.keys() if key != 'power']]
    )

    #mae = np.mean(abs(ground_truth['power'].values - predictions.values))
    #maes += [mae]
    metrics_manager(
        dataset,
        f'{building_name}',
        torch.from_numpy(ground_truth['power'].values).float().view(1,24,1),
        torch.from_numpy(predictions.values).float().view(1,24,1),
        building_types_mask
    )

/projects/foundation/pemami/conda/foundation/lib/python3.9/site-packages/skforecast/utils/utils.py:346: DataTypeWarning: `exog` may contain only `int`, `float` or `category` dtypes. Most machine learning models do not allow other types of values. Fitting the forecaster may fail. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=ValueTypesExogWarning)
  warnings.warn(
/projects/foundation/pemami/conda/foundation/lib/python3.9/site-packages/skforecast/utils/utils.py:346: DataTypeWarning: `exog` may contain only `int`, `float` or `category` dtypes. Most machine learning models do not allow other types of values. Fitting the forecaster may fail. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=ValueTypesExogWarning)
  warnings.warn(
/projects/foundation/pemami/conda/foundation/lib/python3.9/site-packages/skforecast/utils/utils.py:346: DataTypeWarning: `exog` may contain only `int`, `float` or `category` dtypes. Most machine learn

In [16]:
metrics_manager.summary()

,dataset,building_id,building_type,metric,metric_type,value
0,smart,HomeB,residential,cvrmse,scalar,0.569323
1,smart,HomeB,residential,cvrmse_0,hour_of_day,0.342475
2,smart,HomeB,residential,cvrmse_1,hour_of_day,0.508728
3,smart,HomeB,residential,cvrmse_2,hour_of_day,0.639289
4,smart,HomeB,residential,cvrmse_3,hour_of_day,0.595745
...,...,...,...,...,...,...
70,smart,HomeB,residential,nmae_19,hour_of_day,0.262985
71,smart,HomeB,residential,nmae_20,hour_of_day,0.222110
72,smart,HomeB,residential,nmae_21,hour_of_day,0.202935
73,smart,HomeB,residential,nmae_22,hour_of_day,0.216532


In [17]:
bldg_df

,power,hour_x,hour_y,month_x,month_y,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,...,month_of_year_5,month_of_year_6,month_of_year_7,month_of_year_8,month_of_year_9,month_of_year_10,month_of_year_11,month_of_year_12,weekend_False,weekend_True
timestamp,,,,,,,,,,,,,,,,,,,,,
2015-03-06 09:00:00,0.582378,7.071068e-01,-0.707107,1.000000e+00,6.123234e-17,False,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
2015-03-06 10:00:00,0.592038,5.000000e-01,-0.866025,1.000000e+00,6.123234e-17,False,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
2015-03-06 11:00:00,0.626566,2.588190e-01,-0.965926,1.000000e+00,6.123234e-17,False,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
2015-03-06 12:00:00,0.413510,1.224647e-16,-1.000000,1.000000e+00,6.123234e-17,False,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
2015-03-06 13:00:00,0.387672,-2.588190e-01,-0.965926,1.000000e+00,6.123234e-17,False,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-31 20:00:00,0.513937,-8.660254e-01,0.500000,-2.449294e-16,1.000000e+00,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
2016-12-31 21:00:00,0.598407,-7.071068e-01,0.707107,-2.449294e-16,1.000000e+00,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
2016-12-31 22:00:00,0.391207,-5.000000e-01,0.866025,-2.449294e-16,1.000000e+00,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
